In [231]:
import pandas as pd
import numpy as np

In [232]:
# Reading our data stored in a csv
billboard = pd.read_csv('../original/Hot Stuff.csv')
audio_feats = pd.read_excel('../original/Hot 100 Audio Features.xlsx')

In [233]:
billboard.head(3)

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
0,http://www.billboard.com/charts/hot-100/1963-0...,6/1/1963,11,Still,Bill Anderson,StillBill Anderson,1.0,17.0,11.0,8.0
1,http://www.billboard.com/charts/hot-100/1967-0...,1/7/1967,11,Coming Home Soldier,Bobby Vinton,Coming Home SoldierBobby Vinton,1.0,17.0,11.0,8.0
2,http://www.billboard.com/charts/hot-100/1971-0...,7/3/1971,11,She's Not Just Another Woman,The 8th Day,She's Not Just Another WomanThe 8th Day,1.0,17.0,11.0,8.0


In [234]:
audio_feats.head(3)

,SongID,Performer,Song,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_album,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,"AdictoTainy, Anuel AA & Ozuna","Tainy, Anuel AA & Ozuna",Adicto,['pop reggaeton'],3jbT1Y5MoPwEIpZndDDwVq,NaN,Adicto (with Anuel AA & Ozuna),0.0,270740.0,91.0,...,10.0,-4.803,0.0,0.0735,0.017,0.000016,0.179,0.623,80.002,4.0
1,The Ones That Didn't Make It Back HomeJustin M...,Justin Moore,The Ones That Didn't Make It Back Home,"['arkansas country', 'contemporary country', '...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ShallowLady Gaga & Bradley Cooper,Lady Gaga & Bradley Cooper,Shallow,"['dance pop', 'pop']",2VxeLyX666F8uXCJ0dZF8B,NaN,A Star Is Born Soundtrack,0.0,215733.0,88.0,...,7.0,-6.362,1.0,0.0308,0.371,0.000000,0.231,0.323,95.799,4.0


In [235]:
top_ten = billboard[billboard['Week Position'] <= 10]

# Merge top ten music with audio features to get a dataframe with the features of all the top 10 songs
audio_genre = top_ten.merge(right=audio_feats, how='inner', on=['SongID'])

# We transform each year into corresponding decade, eg 1962 => 1960                                                                         
audio_genre['WeekID'] = (pd.to_datetime(audio_genre['WeekID']).dt.year//10)*10

# Transform value representation of genre into list
audio_genre.spotify_genre = audio_genre.spotify_genre.apply(lambda x : str(x).strip('][').split(', '))

# Count number of occurence of each genre among decades
audio_genre = audio_genre.explode("spotify_genre")
df_genre = audio_genre.explode("spotify_genre").groupby("WeekID")["spotify_genre"].value_counts().to_frame('counts')

In [236]:
def main_genre(x):
    if("pop" in x):
        return "pop"
    if("rock" in x):
        return "rock"
    if("rap" in x):
        return "rap"
    if("jazz" in x):
        return "jazz"
    if("r&b" in x):
        return "r&b"
    if("hip hop" in x):
        return "hip hop"
    if("house" in x):
        return "house"
    if("metal" in x):
        return "metal"
    if("country" in x):
        return "country"
    if("folk" in x):
        return "folk"
    if("soul" in x):
        return "soul"
    if("disco" in x):
        return "disco"
    if("blues" in x):
        return "blues"
    if("punk" in x):
        return "punk"
    if("funk" in x):
        return "funk"
    if("electro" in x):
        return "electro"
    return np.nan
    

In [237]:
audio_genre.spotify_genre = audio_genre.spotify_genre.apply(lambda x : main_genre(str(x)))

In [251]:
df_main_genre = audio_genre[['WeekID', 'SongID', 'danceability', 'energy','acousticness', 'loudness','valence', 'tempo','spotify_genre']].dropna().drop_duplicates(subset=['SongID','spotify_genre','WeekID'])

In [252]:
df = audio_genre[['WeekID', 'SongID', 'danceability', 'energy','acousticness', 'loudness','valence', 'tempo','spotify_genre']].dropna().drop_duplicates(subset=['SongID','spotify_genre','WeekID'])

In [253]:
top_decade = df_main_genre.groupby("WeekID")["spotify_genre"].value_counts().to_frame('counts').groupby('WeekID').head(5).reset_index().astype({"WeekID": object})
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(top_decade)

   WeekID spotify_genre  counts
0    1950          rock      67
1    1950           pop      65
2    1950         blues      24
3    1950          soul      18
4    1950          jazz       7
5    1960           pop     565
6    1960          rock     513
7    1960          soul     218
8    1960         blues     115
9    1960          folk      95
10   1970          rock     488
11   1970           pop     338
12   1970          soul     210
13   1970         disco     187
14   1970          funk     146
15   1980          rock     621
16   1980           pop     480
17   1980         disco     138
18   1980          soul      89
19   1980          funk      83
20   1990           pop     361
21   1990           r&b     181
22   1990          rock     150
23   1990       hip hop      92
24   1990          soul      90
25   2000           pop     460
26   2000       hip hop     180
27   2000           r&b     167
28   2000           rap     163
29   2000          rock      50
30   201

In [270]:
df_main_genre = df_main_genre[df_main_genre.WeekID < 2020]
df_rock = df_main_genre[df_main_genre.spotify_genre == "rock"]
df_pop = df_main_genre[df_main_genre.spotify_genre == "pop"]

In [271]:
df_pop_stats = df_pop.groupby(['WeekID','spotify_genre']).mean()
df_rock_stats = df_rock.groupby(['WeekID','spotify_genre']).mean()

In [272]:
max_loud_pop = df_pop_stats.loudness.max()+1
min_loud_pop = df_pop_stats.loudness.min()-1

max_tempo_pop = df_pop_stats.tempo.max()+1
min_tempo_pop = df_pop_stats.tempo.min()-1

max_loud_rock = df_rock_stats.loudness.max()+1
min_loud_rock = df_rock_stats.loudness.min()-1

max_tempo_rock = df_rock_stats.tempo.max()+1
min_tempo_rock = df_rock_stats.tempo.min()-1

In [273]:
df_pop_stats.loudness = df_pop_stats.loudness.apply(lambda x : (x-min_loud_pop)/(max_loud_pop-min_loud))
df_pop_stats.tempo = df_pop_stats.tempo.apply(lambda x : (x-min_tempo_pop)/(max_tempo_pop-min_tempo_pop))

df_rock_stats.loudness = df_rock_stats.loudness.apply(lambda x : (x-min_loud_rock)/(max_loud_rock-min_loud_rock))
df_rock_stats.tempo = df_rock_stats.tempo.apply(lambda x : (x-min_tempo_rock)/(max_tempo_rock-min_tempo_rock))

In [274]:
df_pop_final = df_pop_stats.reset_index()[['WeekID','danceability','energy','acousticness','loudness','valence','tempo']].set_index('WeekID').transpose()
df_rock_final = df_rock_stats.reset_index()[['WeekID','danceability','energy','acousticness','loudness','valence','tempo']].set_index('WeekID').transpose()


In [275]:
df_pop_final

WeekID,1950,1960,1970,1980,1990,2000,2010
danceability,0.554985,0.539358,0.565145,0.649319,0.647526,0.683037,0.671939
energy,0.480898,0.525183,0.548674,0.664241,0.635411,0.700878,0.675083
acousticness,0.656894,0.500841,0.380917,0.208776,0.215376,0.133899,0.148162
loudness,0.082456,0.116015,0.069358,0.135048,0.222827,0.406204,0.402038
valence,0.672692,0.681525,0.652694,0.675296,0.571392,0.591266,0.528508
tempo,0.116623,0.603072,0.427778,0.883377,0.422711,0.459182,0.754848


In [276]:
df_rock_final

WeekID,1950,1960,1970,1980,1990,2000,2010
danceability,0.556418,0.537768,0.563510,0.611659,0.575980,0.564260,0.595880
energy,0.521790,0.562469,0.541313,0.657444,0.640547,0.791260,0.705880
acousticness,0.668853,0.454922,0.355189,0.216617,0.200636,0.069351,0.122007
loudness,0.202504,0.276201,0.127725,0.310780,0.418988,0.872275,0.798447
valence,0.698254,0.692633,0.621395,0.635907,0.533805,0.525868,0.552740
tempo,0.127273,0.222862,0.077315,0.387053,0.135602,0.922685,0.477529


In [104]:
#df_decade = df_decade.transpose()

In [83]:
df_decade

WeekID,1950,1960,1970,1980,1990,2000,2010,2020
danceability,0.544462,0.546660,0.595076,0.643211,0.655540,0.688407,0.688429,0.634591
energy,0.482184,0.532576,0.557135,0.639849,0.615705,0.694357,0.663074,0.640030
acousticness,0.671599,0.498867,0.354096,0.227057,0.222819,0.134022,0.159176,0.285985
loudness,0.000000,0.175414,0.057917,0.270773,0.462423,0.995481,0.977280,1.000000
valence,0.666423,0.684952,0.660680,0.648010,0.556942,0.587202,0.535732,0.539135
tempo,0.472001,0.483870,0.329064,0.520473,0.106411,0.222260,0.505586,1.005443


In [277]:
df_pop_final.to_csv('../data/lollipop_pop.csv')
df_rock_final.to_csv('../data/lollipop_rock.csv')